# Google Maps & Places API
## Set-up

In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
import numpy as np

# Import the API key
from config import g_key

In [2]:
# Load weather data CSV
city_df = pd.read_csv("weather_data/cities.csv")
city_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,New Norfolk,AU,2021-01-12 18:19:40,-42.7826,147.0587,55.00,69,39,1.81
1,1,Salalah,OM,2021-01-12 18:21:16,17.0151,54.0924,75.20,23,0,13.80
2,2,Rikitea,PF,2021-01-12 18:21:16,-23.1203,-134.9692,77.79,77,15,17.38
3,3,Qaanaaq,GL,2021-01-12 18:21:16,77.4840,-69.3632,10.65,83,96,8.23
4,4,Whitehorse,CA,2021-01-12 18:21:17,60.7161,-135.0538,19.40,93,90,3.44


Data used for mapping must be integer or floating-point decimals, so we need to confirm that the weather data to be mapped is in the appropriate format.

In [3]:
# Check datatypes of DataFrame
city_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

Google heatmaps do not plot negative numbers, so we will get an `InvalidWeightException` error if any maximum temperatures are negative. Instead, use a for loop to create a new list of temperatures that takes the maximum of either 0 or the temperature.

In [5]:
# pos_temps = []
# for temp in city_df["Max Temp"]:
#    pos_temps.append(max(temp, 0))

In [37]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_df["Max Temp"]
# Assign the figure variable. Add geographic center of Earth and adjust zoom.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.45)
# Assign the heatmap variable.
## (Using list comprehension to change negative temperatures to 0)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in city_df["Max Temp"]],
                                dissipating=False,
                                max_intensity=400,
                                point_radius=6)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# Heatmap of percent humidity
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.45)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Heatmap of cloudiness
locations = city_df[["Lat", "Lng"]]
clouds = city_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.45)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
# Heatmap of wind speed
locations = city_df[["Lat", "Lng"]]
wind = city_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.45)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=500, point_radius=8)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Get Vacation Criteria

In [47]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [57]:
# Filter the dataset to find cities that meet the inputted temperature criteria
# Drop rows with NaN values using .dropna()
preferred_temp_df = city_df.loc[(city_df["Max Temp"] >= min_temp) & \
                                (city_df["Max Temp"] <= max_temp)].dropna()
preferred_temp_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Salalah,OM,2021-01-12 18:21:16,17.0151,54.0924,75.20,23,0,13.80
2,2,Rikitea,PF,2021-01-12 18:21:16,-23.1203,-134.9692,77.79,77,15,17.38
14,14,Atuona,PF,2021-01-12 18:21:19,-9.8000,-139.0333,77.58,79,46,16.87
17,17,Nikki,BJ,2021-01-12 18:21:20,9.9401,3.2108,83.05,15,0,3.42
26,26,Cape Town,ZA,2021-01-12 18:18:33,-33.9258,18.4232,75.20,69,0,16.11


In [58]:
# Check if the filtered dataset has null rows.
preferred_temp_df.count()

City_ID       150
City          150
Country       150
Date          150
Lat           150
Lng           150
Max Temp      150
Humidity      150
Cloudiness    150
Wind Speed    150
dtype: int64

## Mapping the Criteria

Create heatmaps showing maximum temperature for the filtered cities, including markers indicating the city, country, and the name of a hotel within 3 miles of the coordinates.